In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        img = np.dstack((band_1, band_2, band_3))
        images.append(std_img(img))

    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler,EarlyStopping
def create_model():
    '''Create the FCN and return a keras model.'''

    model=Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3), input_shape=(75, 75, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.3))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.3))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))
    return model
print('model model')


model model


In [5]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def lr_f(epoch):
    if epoch < 15:
        return 0.001
    elif epoch < 30:
        return 0.0005
    elif epoch < 45:
        return 0.0001
    else:
        return 0.00005

def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        early_stopping = EarlyStopping(monitor = 'val_loss', patience = 20, verbose = 0, mode= 'min')
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s,early_stopping]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.57874, saving model to best_m.h5
 - 4s - loss: 0.6661 - acc: 0.6412 - val_loss: 0.5787 - val_acc: 0.7182
Epoch 2/100
Epoch 00002: val_loss did not improve
 - 3s - loss: 0.5372 - acc: 0.7345 - val_loss: 0.6558 - val_acc: 0.6185
Epoch 3/100
Epoch 00003: val_loss did not improve
 - 3s - loss: 0.4435 - acc: 0.7914 - val_loss: 0.7766 - val_acc: 0.6035
Epoch 4/100
Epoch 00004: val_loss did not improve
 - 3s - loss: 0.4556 - acc: 0.7933 - val_loss: 1.0889 - val_acc: 0.5436
Epoch 5/100
Epoch 00005: val_loss improved from 0.57874 to 0.42638, saving model to best_m.h5
 - 3s - loss: 0.4019 - acc: 0.8189 - val_loss: 0.4264 - val_acc: 0.8030
Epoch 6/100
Epoch 00006: val_loss improved from 0.42638 to 0.37562, saving model to best_m.h5
 - 3s - loss: 0.3944 - acc: 0.8276 - val_loss: 0.3756 - val_acc: 0.8130
Epoch 7/100
Epoch 00007: val_loss improved from 0.37562 to 0.32936, saving model to best_m.h5
 - 3s - loss: 0.3869 - acc: 0.8283 - val_loss:

Epoch 63/100
Epoch 00063: val_loss did not improve
 - 3s - loss: 0.2449 - acc: 0.9017 - val_loss: 0.1989 - val_acc: 0.9077
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 3s - loss: 0.2651 - acc: 0.8895 - val_loss: 0.2050 - val_acc: 0.9027
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 3s - loss: 0.2352 - acc: 0.8983 - val_loss: 0.1980 - val_acc: 0.9152
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 3s - loss: 0.2397 - acc: 0.9031 - val_loss: 0.1982 - val_acc: 0.9102
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 3s - loss: 0.2459 - acc: 0.8964 - val_loss: 0.2007 - val_acc: 0.9002
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 3s - loss: 0.2239 - acc: 0.9133 - val_loss: 0.2015 - val_acc: 0.9127
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 3s - loss: 0.2448 - acc: 0.8967 - val_loss: 0.1996 - val_acc: 0.9102
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 3s - loss: 0.2567 - acc: 0.8906 - val_loss: 0.2069 - val_acc: 0.9052
Epoch 71/100
Epo

Epoch 55/100
Epoch 00055: val_loss improved from 0.23171 to 0.23102, saving model to best_m.h5
 - 3s - loss: 0.2157 - acc: 0.9125 - val_loss: 0.2310 - val_acc: 0.9027
Epoch 56/100
Epoch 00056: val_loss improved from 0.23102 to 0.22913, saving model to best_m.h5
 - 3s - loss: 0.2583 - acc: 0.8931 - val_loss: 0.2291 - val_acc: 0.9002
Epoch 57/100
Epoch 00057: val_loss did not improve
 - 3s - loss: 0.2159 - acc: 0.9167 - val_loss: 0.2469 - val_acc: 0.8828
Epoch 58/100
Epoch 00058: val_loss did not improve
 - 3s - loss: 0.2217 - acc: 0.9067 - val_loss: 0.2407 - val_acc: 0.8953
Epoch 59/100
Epoch 00059: val_loss did not improve
 - 3s - loss: 0.2303 - acc: 0.9000 - val_loss: 0.2409 - val_acc: 0.8953
Epoch 60/100
Epoch 00060: val_loss did not improve
 - 3s - loss: 0.2269 - acc: 0.9056 - val_loss: 0.2600 - val_acc: 0.8778
Epoch 61/100
Epoch 00061: val_loss did not improve
 - 3s - loss: 0.2176 - acc: 0.8983 - val_loss: 0.2629 - val_acc: 0.8803
Epoch 62/100
Epoch 00062: val_loss did not improve


Epoch 41/100
Epoch 00041: val_loss did not improve
 - 3s - loss: 0.2671 - acc: 0.8847 - val_loss: 0.2220 - val_acc: 0.8978
Epoch 42/100
Epoch 00042: val_loss did not improve
 - 3s - loss: 0.2948 - acc: 0.8616 - val_loss: 0.2339 - val_acc: 0.8928
Epoch 43/100
Epoch 00043: val_loss did not improve
 - 3s - loss: 0.2686 - acc: 0.8773 - val_loss: 0.2292 - val_acc: 0.9052
Epoch 44/100
Epoch 00044: val_loss did not improve
 - 3s - loss: 0.2590 - acc: 0.8867 - val_loss: 0.2233 - val_acc: 0.8878
Epoch 45/100
Epoch 00045: val_loss did not improve
 - 3s - loss: 0.2499 - acc: 0.8917 - val_loss: 0.2368 - val_acc: 0.8953
Epoch 46/100
Epoch 00046: val_loss did not improve
 - 3s - loss: 0.2149 - acc: 0.9042 - val_loss: 0.2242 - val_acc: 0.8978
Epoch 47/100
Epoch 00047: val_loss did not improve
 - 3s - loss: 0.2634 - acc: 0.8900 - val_loss: 0.2236 - val_acc: 0.8978
Epoch 48/100
Epoch 00048: val_loss did not improve
 - 3s - loss: 0.2615 - acc: 0.8979 - val_loss: 0.2296 - val_acc: 0.8978
Epoch 49/100
Epo

Epoch 15/100
Epoch 00015: val_loss did not improve
 - 3s - loss: 0.3257 - acc: 0.8562 - val_loss: 0.5107 - val_acc: 0.8155
Epoch 16/100
Epoch 00016: val_loss did not improve
 - 3s - loss: 0.3265 - acc: 0.8458 - val_loss: 0.3066 - val_acc: 0.8628
Epoch 17/100
Epoch 00017: val_loss did not improve
 - 3s - loss: 0.2565 - acc: 0.8914 - val_loss: 0.3178 - val_acc: 0.8653
Epoch 18/100
Epoch 00018: val_loss did not improve
 - 3s - loss: 0.2617 - acc: 0.8783 - val_loss: 0.6410 - val_acc: 0.7955
Epoch 19/100
Epoch 00019: val_loss did not improve
 - 4s - loss: 0.2938 - acc: 0.8739 - val_loss: 0.3491 - val_acc: 0.8603
Epoch 20/100
Epoch 00020: val_loss improved from 0.30123 to 0.28371, saving model to best_m.h5
 - 3s - loss: 0.2617 - acc: 0.8892 - val_loss: 0.2837 - val_acc: 0.8828
Epoch 21/100
Epoch 00021: val_loss improved from 0.28371 to 0.28337, saving model to best_m.h5
 - 3s - loss: 0.2668 - acc: 0.8942 - val_loss: 0.2834 - val_acc: 0.8554
Epoch 22/100
Epoch 00022: val_loss did not improve


Epoch 00079: val_loss did not improve
 - 3s - loss: 0.1972 - acc: 0.9242 - val_loss: 0.2591 - val_acc: 0.8878
Epoch 80/100
Epoch 00080: val_loss did not improve
 - 3s - loss: 0.2029 - acc: 0.9100 - val_loss: 0.2712 - val_acc: 0.8853
Epoch 81/100
Epoch 00081: val_loss did not improve
 - 3s - loss: 0.2084 - acc: 0.9092 - val_loss: 0.2625 - val_acc: 0.8928
Epoch 82/100
Epoch 00082: val_loss did not improve
 - 3s - loss: 0.1973 - acc: 0.9192 - val_loss: 0.2646 - val_acc: 0.8928
Epoch 83/100
Epoch 00083: val_loss did not improve
 - 3s - loss: 0.1952 - acc: 0.9233 - val_loss: 0.2617 - val_acc: 0.8878
Epoch 84/100
Epoch 00084: val_loss did not improve
 - 3s - loss: 0.1846 - acc: 0.9192 - val_loss: 0.2879 - val_acc: 0.8803
Epoch 85/100
Epoch 00085: val_loss did not improve
 - 3s - loss: 0.2122 - acc: 0.9183 - val_loss: 0.2687 - val_acc: 0.9002
Epoch 86/100
Epoch 00086: val_loss did not improve
 - 3s - loss: 0.1969 - acc: 0.9233 - val_loss: 0.2693 - val_acc: 0.8878
Epoch 87/100
Epoch 00087: val

In [6]:
import pickle
with open('../features/cnn_5_aug1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_5_aug1_sub.csv', index=False)
# add zoom 0.231
# rm zoom rm flip 0.228
# only ud flip 0.230
# both ud lr flip 0.230

# pre 230
# new 2242

0.224238486867
         id  is_iceberg
0  5941774d    0.161570
1  4023181e    0.888400
2  b20200e4    0.277126
3  e7f018bb    0.998518
4  4371c8c3    0.542188
